In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (125 Ocean Dr # U-0403, Miami Beach, FL 33139,...
2     (445 Grand Bay Dr #1a, Key Biscayne, FL 33149,...
3     (50 South Pointe Dr 1902 03, Miami Beach, FL 3...
4     (9701 Collins Ave #803s, Miami Beach, FL 33154...
5     (7600 Fisher Island Dr APT 7622, Miami Beach, ...
6     (10 Edgewater Dr #16a, Miami, FL 33133, USA, (...
7     (18501 Collins Ave #1003, Sunny Isles Beach, F...
8     (Santa Maria, 1643 Brickell Ave APT 3804, Miam...
9      (Miami Beach, FL, USA, (25.790654, -80.1300455))
10    (1 Collins Ave Unit 404, Miami Beach, FL 33139...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Ocean House\r125 Ocean Dr U-0403\rMiami Beach,1/5/2022,3/7/2022,41,"$16,500,000.00",4365,"$3,780.07",Pablo Alfaro PA,Lisa Van Wagenen,Ocean House 125 Ocean Dr U-0403 Miami Beach,Ocean House,125 Ocean Dr U-0403 Miami Beach,"(125 Ocean Dr # U-0403, Miami Beach, FL 33139,...","(25.7700548, -80.1321744, 0.0)",25.770055,-80.132174,0.0
2,Grand Bay\r445 Grand Bay Dr PH-1A\rKey Biscayne,10/9/2021,3/9/2022,55,"$11,375,000.00",11260,"$1,348.55",Brigitte Nachtigall,Barbara Lamar PA,Grand Bay 445 Grand Bay Dr PH-1A Key Biscayne,Grand Bay,445 Grand Bay Dr PH-1A Key Biscayne,"(445 Grand Bay Dr #1a, Key Biscayne, FL 33149,...","(25.6926193, -80.1589937, 0.0)",25.692619,-80.158994,0.0
3,Continuum North\r50 S Pointe Dr 1902-03\rMiami...,8/26/2021,3/7/2022,163,"$9,800,000.00",3403,"$2,879.81",Sheila Rojas,M Claudia Hersman,Continuum North 50 S Pointe Dr 1902-03 Miami B...,Continuum North,50 S Pointe Dr 1902-03 Miami Beach,"(50 South Pointe Dr 1902 03, Miami Beach, FL 3...","(25.767875, -80.132756, 0.0)",25.767875,-80.132756,0.0
4,St Regis Bal Harbour\r9701 Collins Ave 803S\rB...,11/11/2021,3/7/2022,83,"$5,900,000.00",2840,"$2,077.46",Roman Sinyavsky,Silvia Olivera PA,St Regis Bal Harbour 9701 Collins Ave 803S Bal...,St Regis Bal Harbour,9701 Collins Ave 803S Bal Harbour,"(9701 Collins Ave #803s, Miami Beach, FL 33154...","(25.8884913, -80.1228812, 0.0)",25.888491,-80.122881,0.0
5,Oceanside\r7600 Fisher Island Dr 7622\rFisher ...,2/6/2022,3/7/2022,2,"$5,750,000.00",2861,"$2,009.79",Monica Harvey,Robert Vole,Oceanside 7600 Fisher Island Dr 7622 Fisher Is...,Oceanside,7600 Fisher Island Dr 7622 Fisher Island,"(7600 Fisher Island Dr APT 7622, Miami Beach, ...","(25.7612272, -80.1363805, 0.0)",25.761227,-80.136381,0.0
6,The Gables Club Condo\r10 Edgewater Dr 16A\rCo...,1/6/2022,3/7/2022,14,"$5,500,000.00",3830,"$1,436.03",Allison Blumenthal,Ilana Levitt,The Gables Club Condo 10 Edgewater Dr 16A Cora...,The Gables Club Condo,10 Edgewater Dr 16A Coral Gables,"(10 Edgewater Dr #16a, Miami, FL 33133, USA, (...","(25.706719, -80.25356099999999, 0.0)",25.706719,-80.253561,0.0
7,Turnberry Ocean Club\r18501 Collins Ave 1003\r...,1/7/2022,3/7/2022,17,"$4,550,000.00",3760,"$1,463.02",Tamara Orfali,Mariana Niro,Turnberry Ocean Club 18501 Collins Ave 1003 Su...,Turnberry Ocean Club,18501 Collins Ave 1003 Sunny Isles Beach,"(18501 Collins Ave #1003, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
8,Santa Maria\r1643 Bricjell Ave 3804\rMiami,11/12/2021,3/9/2022,55,"$3,850,000.00",4030,$955.34,Fabian Dominguez,Fabian Dominguez,Santa Maria 1643 Bricjell Ave 3804 Miami,Santa Maria,1643 Bricjell Ave 3804 Miami,"(Santa Maria, 1643 Brickell Ave APT 3804, Miam...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
9,Monad Terrace\r1300 Monad Terr 4B\rMiami Beach,12/6/2021,3/10/2022,18,"$3,800,000.00",2667,"$1,424.82",Randal Longo,Reid Heidenry,Monad Terrace 1300 Monad Terr 4B Miami Beach,Monad Terrace,1300 Monad Terr 4B Miami Beach,"(Miami Beach, FL, USA, (25.790654, -80.1300455))","(25.790654, -80.1300455, 0.0)",25.790654,-80.130045,0.0
10,One Ocean\r1 Collins Ave 404\rMiami Beach,5/7/2021,3/11/2022,263,"$3,650,000.00",2486,"$1,468.22",Anamaria Velasquez,Jonathan Corso,One Ocean 1 Collins Ave 404 Miami Beach,One Ocean,1 Collins Ave 404 Miami Beach,"(1 Collins Ave Unit 404, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 6th - March 12th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')